# Testing

Testing is extremely important. Without testing, you cannot be sure that your code is doing what you think. Testing is an integral part of software development, and should be done *while* you are writing code, not after the code has been written.

No doubt sofar, you have been manually checking that your code does the right thing. Perhaps you are tunning your code over a particular input file and making sure that you get a correct-looking plot out at the end. This is a start but how can you be sure that there's not a subtle bug that means that the output is incorrect? And if there *is* a problem, how will you be able to work out exactly which line of code it causing it?

In order to be confident that our code it giving a correct output, a *test suite* is useful which provides a set of known inputs and checks that the code matches a set of known, expected outputs. To make it easier to locate where a bug is occuring, it's a good idea to make each individual test run over as small an amount of code as possible so that if *that* test fails, you know where to look for the problem. In Python this "small unit of code" is usually a function.

Let's get started by making sure that our `add_arrays` function matches the outputs we expect.

Since we are writing some tests for our `arrays` module, let's make a file called `test_arrays.py` which contains the following:

In [1]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    if output == expect:
        print("OK")
    else:
        print("BROKEN")

test_add_arrays()

Overwriting test_arrays.py


This script defines a function called `test_add_arrays` which defines some known input (`a` and `b`) and a known, matching output (`expect`). It passes them to the function `add_arrays` and compares the output to `expected`. It will either print `OK` or `BROKEN` depending on whether it's working or not. Finally, we explicitly call the test function.

In [2]:
# TODO Make this appear to the reader as `python test_arrays.py` and hide the "In []"

%run test_arrays.py

OK


### Exercise

Break the test by changing either `a`, `b` or `expected` and rerun the test script. Make sure that it prints `BROKEN` in this case. Change it back to a working state once you've done this.

... Change it to use an assert.

In [3]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

test_add_arrays()

Overwriting test_arrays.py


..now when we run we get nothing printed on success:

In [4]:
%run test_arrays.py

...but on a failure we get an error printed like:

```
Traceback (most recent call last):
  File "test_arrays.py", line 13, in <module>
    test_add_arrays()
  File "test_arrays.py", line 11, in test_add_arrays
    assert output == expect
AssertionError
```

which, if we had many test functions being run would tell us which one failed and on which line.

...We want to automate the runnng of these tests and there is a tool called `pytest` which does this.

Remove the call to `test_add_arrays()` on the last line of the file:

In [5]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


And run `pytest`:

In [6]:
!COLUMNS=80 venv/bin/pytest

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 1 item                                                               

test_arrays.py .                                                         [100%]

============================== 1 passed in 1.50s ===============================


...If we break the test then we see...



...

### Exercise